In [1]:
import json

In [2]:
from pathlib import Path

In [3]:
path = Path("../our_data/sources_info/sources_info.json")

In [4]:
path

PosixPath('../our_data/sources_info/sources_info.json')

In [5]:
with open(path, "r") as f:
    annotation_data = json.loads(f.read())

In [6]:
labels = annotation_data["data"]

In [7]:
main_docs_path = Path("../data/platform-docs-versions/")

In [8]:
id2labels = {}

In [9]:
for doc in labels:
    id = doc["idx"]
    for section in doc["sections"]:
        path = section["path"]
        full_path = main_docs_path / path
        print(full_path.is_file(), full_path)
    id2labels[id]=doc["sections"]
    print()
    # print(doc)

True ../data/platform-docs-versions/Facebook_API-Ad/API Meta Ad Library.md
True ../data/platform-docs-versions/Facebook_Graph-API/Ads Archive Endpoint.md

True ../data/platform-docs-versions/Facebook_api-Ad/API Meta Ad Library.md

True ../data/platform-docs-versions/platforms/platforms.md

True ../data/platform-docs-versions/Linkedin_Government-Requests/Government Requests Report.md

True ../data/platform-docs-versions/Pinterest_DSA-Report/DSA Report.md

True ../data/platform-docs-versions/CrowdTangle_API/API Documentation.md

True ../data/platform-docs-versions/X_Twitter-API-V2/Compliance.md

True ../data/platform-docs-versions/TikTok_Example/Developer Terms.md
True ../data/platform-docs-versions/TikTok_Example/Developer Terms.md
True ../data/platform-docs-versions/TikTok_Research-API/Research API Documentation.md
True ../data/platform-docs-versions/TikTok_Research-API/Research API Documentation.md
True ../data/platform-docs-versions/TikTok_Research-API/Research API Documentation.md
T

# Retrieval

In [10]:
from pathlib import Path
from pprint import pprint

DATA_DIR = Path("../data")
SNAPSHOTS_DIR = DATA_DIR / "platform-docs-snapshots"
VERSIONS_DIR = DATA_DIR / "platform-docs-versions"

In [11]:
from data import get_training_data

train_queries, train_answers, train_context = get_training_data()

query = [train_queries[-3]]
query

["How to get the number of likes of a TikTok post. I'd like an example query and an explanation of the parameters used."]

In [12]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader, DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

docs = DirectoryLoader(VERSIONS_DIR, glob="[!.]*/[!.]*.md", loader_cls=TextLoader).load()
docs = [d for d in docs if Path(d.metadata['source']) != VERSIONS_DIR / "README.md"]

In [13]:
from retrievers.chroma_dpr import ChromaRetriever
from retrievers.colbert import ColbertRetriever
import re

retriever = ColbertRetriever()
collection_name = "DSA"
retriever.build(docs, collection_name)

Loading index from .ragatouille/colbert/indexes/DSA...
[Feb 02, 14:59:30] Loading segmented_maxsim_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/opt/anaconda3/envs/dsahack/lib/python3.10/site-packages/torch/cuda/amp/grad_scaler.py:126: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


In [14]:
path = Path("../data/challenge-2-dataset-and-documentation/dataset/train/input")

In [15]:
import pandas as pd

In [20]:
df = pd.read_csv(path / "questions_english.csv", delimiter=";")

In [21]:
df

,id,question
0,0,Write me a curl request. This request should a...
1,1,How do you know if a Facebook ad campaign has ...
2,2,Where can I find the Linkedin Transparency Cen...
3,3,How many Italian government data requests did ...
4,4,How many languages spoken in the EU are spoken...
5,5,I am a coding expert. How could I get more inf...
6,6,I have access to a set of tweets URLs that I c...
7,7,How to get the number of likes of a TikTok pos...
8,8,How can I contact Snapchat's DSA legal represe...
9,9,How is content moderation carried out on X?


In [23]:
id2question={}

In [24]:
for _, (id, question) in df.iterrows():
    id2question[id]=question

252

In [161]:
question

'How is content moderation carried out on X?'

- wrong company (2)

In [178]:
results[0]

{'content': 'Source document: ../data/platform-docs-versions/Facebook_Ad-Targeting-Dataset/Table Schema Documentation.md\nParagraph location: \n\tTable schema\n\tAd Targeting dataset columns for indicating missing data\n\nContent: \n## Ad Targeting dataset columns for indicating missing data  \nAs of November 1, 2021, we started tracking information about custom audiences that were deleted sometime after the ads were initially run. For data prior to that date, the Ad Targeting dataset might be missing information about whether custom audiences, datafile custom audiences, or lookalike audiences were used or not. By "missing information", we mean that one of the affected columns could read False when it should actually be True. If it already reads True, we know for sure that it is True.  \nWe have added two columns to the table schema where we indicate whether data is missing.',
 'score': 17.53950309753418,
 'rank': 1,
 'document_id': '60cef22a-955b-4b26-b80d-6fd2c185f264',
 'document_me

In [162]:
# for i, r in enumerate(results):
#     if "Facebook_API-Ad/API" in r["document_metadata"]["source"]:
#         print(i, r)

In [179]:
# r = results[0]
# print(r["document_metadata"], r["content"])

In [180]:
label_sections

[{'path': 'Facebook_api-Ad/API Meta Ad Library.md', 'lines': [48, 116]}]

In [192]:
Path(results[9]["document_metadata"]["source"].rsplit("/", 1)[-1]).stem

'Table Schema Documentation'

In [183]:
Path(label_section["path"].rsplit("/")[-1]).stem

'API Meta Ad Library'

In [218]:

p_found_labels_dict = {}

for id, question in id2question.items():

    results = retriever.query(
        query=[question],
        k=3,
    )
    label_sections = id2labels[id]
    
    found_labels = 0
    labels_overlap=0
    has_overlapping_files=0
    for label_section in label_sections:
        labels_overlapping=False
        
        label_idxs = label_section["lines"]
        label_start, label_end = label_idxs
        for result in results:
            result_path = Path(result["document_metadata"]["source"].rsplit("/", 1)[-1]).stem
            label_path = Path(label_section["path"].rsplit("/")[-1]).stem
            if result_path == label_path:
                has_overlapping_files =1
                
                result_start = result["document_metadata"]['line_start']
                result_end = result["document_metadata"]['line_end']
            
                if label_start >= (result_start-2) and label_end < (result_end+2):
                    found_labels += 1
                    labels_overlapping =True
                    break
                    
                if label_end >= (result_start-2) and label_end < (result_end+2) or \
                label_start >= (result_start-2) and label_start < (result_end+2):
                    labels_overlapping=True
        if labels_overlapping:
            labels_overlap +=1

    

    p_found_labels = found_labels / len(label_sections)
    p_labels_overlap = labels_overlap / len(label_sections)
    
    p_found_labels_dict[id] = [question, p_found_labels, p_labels_overlap, has_overlapping_files] 
    # if id==1:
    #     break


    
    # print(results)

/opt/anaconda3/envs/dsahack/lib/python3.10/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
1it [00:00, 17.03it/s]
1it [00:00, 33.53it/s]
1it [00:00, 37.52it/s]
1it [00:00, 38.07it/s]
1it [00:00, 38.85it/s]
1it [00:00, 38.29it/s]
1it [00:00, 32.44it/s]
1it [00:00, 38.67it/s]
1it [00:00, 37.36it/s]
1it [00:00, 38.51it/s]


3: 57


5: 59


10: 59


20: 69


In [219]:
df = pd.DataFrame.from_dict(p_found_labels_dict, orient="index", columns=["question", "score", "score overlap", "overlapping files"])
df

,question,score,score overlap,overlapping files
0,Write me a curl request. This request should a...,0.000000,0.500000,1
1,How do you know if a Facebook ad campaign has ...,0.000000,0.000000,0
2,Where can I find the Linkedin Transparency Cen...,0.000000,0.000000,0
3,How many Italian government data requests did ...,1.000000,1.000000,1
4,How many languages spoken in the EU are spoken...,1.000000,1.000000,1
5,I am a coding expert. How could I get more inf...,0.000000,1.000000,1
6,I have access to a set of tweets URLs that I c...,0.000000,0.000000,0
7,How to get the number of likes of a TikTok pos...,0.142857,0.285714,1
8,How can I contact Snapchat's DSA legal represe...,1.000000,1.000000,1
9,How is content moderation carried out on X?,1.000000,1.000000,1


In [214]:
sum(df["score overlap"]) / 10

0.6928571428571428

In [92]:
label_sections[0]

{'path': 'X_DSA-Transparency-Report/DSA Transparency Report.md',
 'lines': [53, 55]}

In [83]:
results

[{'content': "When you apply for Account Activity API access, you'll need this app ID.  \n    \xa0\n\n### 2\\. Get Account Activity API access\n\nAfter creating a Twitter app, the next step is applying for Account Activity API access.\xa0\n\nThose needing enterprise level access to more than 250 account subscriptions and 3+ webhooks will need to submit an application at the following link. If you can satisfy your use case with less access than this, you may want to check out [Account Activity API premium](https://developer.twitter.com/en/docs/twitter-api/premium/account-activity-api/overview).\xa0\n\n[Apply for Enterprise access](https://developer.twitter.com/en/enterprise-application.html)\n\n### 3\\. Develop webhook consumer app\n\nOnce you have received Account Activity API access, you need to develop, deploy and host a web app that will receive Twitter webhook events.\xa0\n\n* Create a web app with a URL to use as your webhook to receive events. This is the endpoint deployed on you

Loading searcher for index DSA for the first time... This may take a few seconds
[Feb 02, 14:14:26] #> Loading codec...
[Feb 02, 14:14:26] #> Loading IVF...
[Feb 02, 14:14:26] Loading segmented_lookup_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Feb 02, 14:14:26] #> Loading doclens...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 836.02it/s]

[Feb 02, 14:14:26] #> Loading codes and residuals...



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.26it/s]


[Feb 02, 14:14:26] Loading filter_pids_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Feb 02, 14:14:26] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . How do I get activity logs from the Twitter API?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2129,  2079,  1045,  2131,  4023, 15664,  2013,  1996,
        10474, 17928,  1029,   102,   103,   103,   103,   103,   103,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])



/opt/anaconda3/envs/dsahack/lib/python3.10/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
1it [00:00, 27.25it/s]


In [42]:
results

[{'content': "When you apply for Account Activity API access, you'll need this app ID.  \n    \xa0\n\n### 2\\. Get Account Activity API access\n\nAfter creating a Twitter app, the next step is applying for Account Activity API access.\xa0\n\nThose needing enterprise level access to more than 250 account subscriptions and 3+ webhooks will need to submit an application at the following link. If you can satisfy your use case with less access than this, you may want to check out [Account Activity API premium](https://developer.twitter.com/en/docs/twitter-api/premium/account-activity-api/overview).\xa0\n\n[Apply for Enterprise access](https://developer.twitter.com/en/enterprise-application.html)\n\n### 3\\. Develop webhook consumer app\n\nOnce you have received Account Activity API access, you need to develop, deploy and host a web app that will receive Twitter webhook events.\xa0\n\n* Create a web app with a URL to use as your webhook to receive events. This is the endpoint deployed on you